# `singledispatch` notes

Some brief notes on experimenting with `singledispatch` in Python. 

* [official documentation](https://docs.python.org/3/library/functools.html#functools.singledispatch)
* [Python 2.6–3.3 backport](https://pypi.python.org/pypi/singledispatch)

In [1]:
from functools import singledispatch

### “Hello, `singledispatch`!”

Here is a simple example that provides a base implementation that expects a string and an alternate handler for integers:

In [2]:
@singledispatch
def hello(a_str):
    print('Hello, {}!'.format(a_str))
    
@hello.register(int)
def _(an_int):
    print('Oooo, a whole number! ({})'.format(an_int))
    
hello('world')
hello(10)

Hello, world!
Oooo, a whole number! (10)


### `singledispatch` “super”

Does `singledispatch` support calling from one registered function to another? A typical use case is handling a single element versus a list, which I'd normally spell like this:

```python
def process_list(list_or_str):
    if isinstance(list_or_str, basestring):
        list_or_str = [list_or_str]

    # rest of function
```

With `singledispatch`, the typecheck can be removed and the preamble eliminated from the main implementation. Here's a silly example that prints the length of a list, which interprets strings as single-element lists:

In [3]:
@singledispatch
def list_len(a_list):
    # in case of a real function, this block would 
    # be something interesting

    print(len(a_list))    

@list_len.register(str)
def _(a_str):
    return list_len([a_str])


This spelling promotes a nice separation of concerns: the main implementation can focus on the actual task at hand, while the chore of smoothing out call argument types — in this instance, the difference between being called with a list or a string — can be handled by registered handlers for the types you'd like to accept, each of which normalizes its input to the “base” implementation's signature and delegates the work to it. Here is an example of calling it:

In [4]:
list_len([0, 1, 2, 3, 4])

5


In [5]:
# treat a string as a single element list rather than as a sequence
list_len('a string')

1


### `singledispatch` and different signatures

As expected in a free-form language like Python, the argument signatures of registered functions do not have to match as there is no checking performed. A use case for providing different signatures is to have an implementation that accepts an argument list:

In [6]:
@singledispatch
def connect(host, port=80):
    print('Opening connection to {}:{}'.format(host, port))
    
connect('python.org')

Opening connection to python.org:80


but also offer an override that accepts a “packed” object — either a class instance or a generic container like `dict` or `list` — which is unpacked and sent to the base implementation. Here is an example that unpacks a `tuple` and invokes `connect()` using the `tuple`'s values as arguments:

In [7]:
@connect.register(tuple)
def _(a_tuple):
    return connect(*a_tuple)

connect(('python.org',))
connect(('python.org', 443))

Opening connection to python.org:80
Opening connection to python.org:443
